# [프로젝트] 멀티 에이전트 기반 Research Agent 만들기

STORM, Open Deep Research의 구조에 착안하여, 적절한 로직을 세우고 작동하는 Research Agent를 만들어 보겠습니다.

In [ ]:
!pip install langgraph dotenv arxiv langchain-tavily langchain-community langchain-google-genai pymupdf -q

In [ ]:
from dotenv import load_dotenv
import os

# GOOGLE_API_KEY, TAVILY_API_KEY 필수
load_dotenv()

LangSmith 설정

In [ ]:
os.environ['LANGCHAIN_API_KEY'] = ''
os.environ['LANGCHAIN_PROJECT'] = 'LangGraph_FastCampus'
os.environ['LANGCHAIN_ENDPOINT'] = 'https://api.smith.langchain.com'
os.environ['LANGCHAIN_TRACING_V2']='true'

## Preliminary

이번 실습에서는, 그동안 사용하지 않았던 랭체인의 특별한 기능들을 더 활용해 보겠습니다.

#### 다중 LLM 사용하기

LangChain의 `init_chat_model`은 provider와 model 정보를 입력하는 방식으로   
다양한 모델을 불러올 수 있는 기능입니다.

In [ ]:
import os
from langchain_core.rate_limiters import InMemoryRateLimiter
from langchain.chat_models import init_chat_model
from rich import print as rprint

# Gemini API는 분당 10개 요청으로 제한
# 즉, 초당 약 0.167개 요청 (10/60)
rate_limiter = InMemoryRateLimiter(
    requests_per_second=0.167,  # 분당 10개 요청
    check_every_n_seconds=0.1,  # 100ms마다 체크
    max_bucket_size=10,  # 최대 버스트 크기
)

quick_rate_limiter = InMemoryRateLimiter(
    requests_per_second=0.333,  # 분당 20개 요청
    check_every_n_seconds=0.1,  # 100ms마다 체크
    max_bucket_size=20,  # 최대 버스트 크기
)



llm = init_chat_model(
    model_provider="google_genai",
    model="gemini-2.0-flash",
    # model = "gemini-2.0-flash-lite"
    # model = "gemini-2.0-flash-thinking-exp"
    rate_limiter=rate_limiter,
    temperature=0.8,
)

rprint(llm)

## Configurables   

LangGraph 어플리케이션의 작동을 조율하기 위해, State와 함께 `Configurable`를 추가할 수 있습니다.


In [ ]:
from langchain_core.runnables.config import RunnableConfig
from typing_extensions  import TypedDict, Optional

class Configuration(TypedDict):

    # Search Metadata
    num_search_queries: int
    max_search_depth: int

    # Models
    planner_model:str
    planner_provider:str

    writer_model:str
    writer_provider:str

    evaluator_model: str
    evaluator_provider: str

    quick_model: str
    quick_provider: str

    search_api=list[str]

# Default

default = Configuration(
    num_search_queries=3,
    max_search_depth=2,

    planner_model='models/gemini-2.0-flash',
    planner_provider='google-genai',

    writer_model='models/gemini-2.0-flash',
    writer_provider='google-genai',

    evaluator_model='models/gemini-2.0-flash',
    evaluator_provider='google-genai',

    quick_model='models/gemini-2.0-flash-lite',
    quick_provider='google-genai',



    search_api=['tavily'] # 이후 다른 검색 엔진 추가
)



#### 검색 API 활용하기   
이번 프로젝트에서는 2개의 검색 API를 활용합니다.

1. Tavily Search
3. Arxiv Search

추가적인 검색 엔진으로는
PubMed 등의 도메인 특화 검색이나, Perplexity, Exa 등의 유료 API를 연결할 수 있습니다.

Tavily Search를 구성합니다.   
만약 모듈의 Argument를 LLM이 판단하게 하고 싶다면,   
해당 함수를 툴로 감싸는 식으로 만들면 됩니다.

In [ ]:
# Tavily Search

from langchain_tavily import TavilySearch

tavily_search = TavilySearch(
    max_results=5,
    topic="general",
    # include_answer=False,
    include_raw_content=True,
    # include_images=False,
    # include_image_descriptions=False,
    # search_depth="basic",
    # time_range="day",
    # include_domains=None,
    # exclude_domains=None
)

In [ ]:
result = tavily_search.invoke("Retrieval Augmented Generation Reasoning")
len(result)


논문과 테크 리포트를 검색하는 학술 검색 API입니다.

In [ ]:
from langchain_community.retrievers import ArxivRetriever

arxiv_search = ArxivRetriever(
    load_max_docs=5,
    load_all_available_meta=True,
    get_full_documents=True,
    doc_content_chars_max= 100000
    # 10만 글자까지만 수집

)

In [ ]:
docs = arxiv_search.invoke("Retrieval Augmented Generation Reasoning")
# docs

In [ ]:
for doc in docs:
    print(f"Published: {doc.metadata['Published']}")
    print(f"Title: {doc.metadata['Title']}")
    print(f"Authors: {doc.metadata['Authors']}")
    print(f"Summary: {doc.metadata['Summary']}")
    print(f"Length: {len(doc.page_content)}")
    print("-" * 50)



각각의 검색 API를 아래와 같이 정리해 놓겠습니다.

In [ ]:
tool_list = {
    'tavily': tavily_search,
    'arxiv': arxiv_search,
}

전체 과정은 다음과 같이 이루어집니다.

1) 연구 토픽을 입력하면, LLM이 추가 정보를 질문합니다.    
유저는 그대로 진행하거나, 피드백을 전달합니다.

2) 연구 토픽에 대해, LLM이 간단한 검색을 수행하고 이를 바탕으로 연구 개요를 작성합니다.   

3) 개요에 포함된 각 세션에 대해, LLM이 검색 쿼리를 생성하여 각 검색엔진을 통해 검색합니다.   

4) 검색된 결과를 바탕으로 섹션별 내용을 작성합니다.    
(검색 결과에 대한 레퍼런스 표시를 포함합니다.)

5) 섹션별 드래프트를 개선하기 위해, 파생 질문을 추가로 생성하여 더 검색하거나 작성을 종료합니다.


6) 섹션별 내용을 취합하고, 최종 수정을 거친 뒤 리포트를 완성합니다.



In [ ]:
from langchain.prompts import PromptTemplate, ChatPromptTemplate
from langchain_core.output_parsers import PydanticOutputParser, StrOutputParser
from langgraph.types import Command, interrupt
from langgraph.graph import StateGraph, START, END
from langgraph.checkpoint.memory import MemorySaver
from pydantic import BaseModel, Field
from typing_extensions import Annotated, Literal
import operator
from langgraph.constants import Send

## 작업에 사용할 클래스 만들기   
Structured Output을 위한 클래스를 먼저 구성합니다.

In [ ]:
class Section(BaseModel):
    name: str = Field(description="섹션의 이름")
    description: str = Field(description="해당 섹션에서 다룰 주요 주제에 대한 간략한 개요")
    content: str = Field(description="섹션의 내용 (처음에는 비워 둡니다)")

    @property
    def as_str(self) -> str:
        """섹션의 정보를 포맷팅된 문자열로 변환합니다."""
        return f"### {self.name}\n{self.description}\n\n내용:\n{self.content}"

class ReportPlan(BaseModel):
    sections: list[Section] = Field(description="A list of sections for the report.")
    followup_question: str = Field(description="사용자에게 추가로 질문할 내용 (없으면 '')")

    @property
    def as_str(self) -> str:
        """섹션들을 포맷팅된 문자열로 변환합니다."""
        sections_str = []
        for section in self.sections:
            sections_str.append(f"### {section.name}\n{section.description}")
        return "\n\n".join(sections_str)


## 서브모듈: 토픽에 대한 리서치 모듈 만들기

섹션의 개요가 주어지면, 해당 내용을 검색하여 섹션을 작성하는 과정을 구현해 보겠습니다.

In [ ]:
class ResearchState(TypedDict):
    topic: str
    section: Section
    queries: list[str]
    draft: str
    resources: list
    num_revision: int
    finished: bool


def generate_search_query(state: ResearchState, config: RunnableConfig):
    prompt = ChatPromptTemplate([
('system', f'''
주어진 섹션 정보에 대한 사전 조사를 위해, 효과적인 검색 쿼리를 생성해야 합니다.
해당 주제를 포괄적으로 다룰 수 있는 검색 쿼리들을 생성하세요.

검색 쿼리는 다음과 같은 원칙을 따라야 합니다:
1. 핵심 키워드를 포함해야 합니다
2. 너무 일반적이지 않아야 합니다
3. 학술적이고 전문적인 용어를 사용해야 합니다
4. 최신 연구 동향을 반영해야 합니다
5. 따옴표를 포함하지 않아야 합니다.

적절한 검색 쿼리를 한 줄에 하나씩 작성하세요.
쿼리만 출력하고, {config['configurable']['num_search_queries']} 개의 쿼리를 출력하세요.

'''),
('user', '''
섹션 정보:
{section}

''')])
    section = state['section']

    writer_llm = init_chat_model(
        model_provider = config['configurable']['writer_provider'],
        model= config['configurable']['writer_model'],
        rate_limiter=rate_limiter,
        temperature=0.8,
        max_tokens = 8192
    )


    chain = prompt | writer_llm | StrOutputParser() | (lambda x: x.split('\n'))

    queries = chain.invoke(section.as_str)

    return {'queries':queries}

def search_and_filter(state: ResearchState, config: RunnableConfig):
    '''각각의 검색어에 대해 검색 결과를 수행하고 필터링합니다.
    Tavily Search는 정확성이 높기 때문에, 해당 부분을 생략해도 됩니다..
    빠른 LLM을 사용하겠습니다.
    '''
    quick_llm = init_chat_model(
        model_provider = config['configurable']['quick_provider'],
        model= config['configurable']['quick_model'],
        rate_limiter=quick_rate_limiter,
        temperature=0.8,
    )

    search_tool =  tool_list[config['configurable']['search_api'][0]]
    # 적절한 검색 툴 선택 (여기서는 Tavily로 고정)

    queries = state['queries']
    section = state['section']

    relevant_docs=[]


    filter_prompt=ChatPromptTemplate([
        ('system', f'''다음 검색 결과가 주어진 주제와 관련이 있는지 O/X로 판단하세요.
O/X만 출력하세요.
---
주제: {section.as_str}'''),

('user', '''
검색 결과: {doc}''')])
    chain = filter_prompt | quick_llm | StrOutputParser()

    context = search_tool.batch(queries)

    def preprocess(text):
        import re
        # 탭과 개행문자를 공백으로 변환
        text = text.replace('\t', ' ').replace('\n', ' ').replace('\xa0', ' ')
        # 템플릿 오류 방지
        text = text.replace('{', '(').replace('}', ')')

        # 연속된 공백을 하나로 치환
        text = re.sub(r'\s+', ' ', text).strip()
        return text


    for docs in context:
        try:
            for doc in docs['results']:
                doc_str = f"### {doc['title']} \n URL: {doc['url']} \n {doc['raw_content']}" if doc.get('raw_content') else f"### {doc['title']} \n {doc['content']}"
                # 하나로 만든 뒤 전처리
                doc_str = preprocess(doc_str)
                relevance = chain.invoke(doc_str)
                if relevance=='O':
                    relevant_docs.append(doc_str)
        except: # 검색 오류시
            continue
    print(f'# Filtered Docs: {len(relevant_docs)}')
    return {'resources':relevant_docs}

def write_section(state: ResearchState, config: RunnableConfig):
    section = state['section'].as_str
    topic = state['topic']
    resources = state['resources']

    writer_prompt =ChatPromptTemplate([
        ('system', '''
연구 리포트의 주제와 세부 섹션명이 주어집니다.
아래의 정보를 활용하여, 연구 리포트의 한 섹션을 작성하세요.
다음은 작성 가이드라인입니다.


[작성 가이드라인]
간단하고 명확한 언어를 사용하세요.
섹션명은 마크다운 ## 으로 작성하며, 세부 목차는 만들지 말고 문단으로만 분리하세요.
문장을 너무 길게 쓰지 말고, 이해하기 쉽게 작성하세요.
'이다.' 가 아닌 '입니다.', '합니다.' 등의 스타일로 작성하세요.
또한, 아래에 주어지는 정보의 내용을 최대한 활용하여 작성하세요.


[인용 가이드라인]
인용 표시는 [1], [2]와 같이 작성하고, 섹션 마지막에 레퍼런스를 작성하세요.
레퍼런스 형식은 MLA 표기를 따르고, 마지막에 URL도 표시하세요.
예시 표시 형식은 다음과 같습니다.

**References**
[1] Unite.AI. "DeepMind의 Michelangelo 벤치마크: Long-Context LLM의 한계를 드러내다." *Unite.AI*, [https://www.unite.ai/ko/
<br><br>
[2] ...


[노트]
인용 표시를 정확하게 했는지 확인하고, 주장이 기술되는 경우 가급적 소스에 근거하도록 작성하세요.
마크다운 형식을 고려하여, 문단 분리나 레퍼런스 사이의 줄바꿈을 명확하게 하세요.

[기존 드래프트]

기존 드래프트가 주어지는 경우, 여기에 이어서 작성하세요.
'''),

('user',f'''
주제: {topic}

세부 섹션명: {section}

검색 결과 Context:
{resources}
''')
])
    writer_llm = init_chat_model(
        model_provider = config['configurable']['writer_provider'],
        model= config['configurable']['writer_model'],
        rate_limiter=rate_limiter,
        temperature=0.8,
        max_tokens = 8192
    )
    chain = writer_prompt | writer_llm | StrOutputParser()
    draft = chain.invoke({})
    return {'draft':draft}




class Feedback(BaseModel):
    grade : Literal['Good', 'Bad'] = Field(description='드래프트에 대한 평가')
    queries: Optional[list[str]] = Field(description='검색 쿼리 목록')

def refine_research(state: ResearchState, config: RunnableConfig) -> Command[Literal[END, "search_and_filter"]] :
    '''Draft와 context를 평가하여, 추가 검색이 필요한지 판단합니다.
    Revision 개수를 초과하면 바로 END로 이동합니다.'''

    section = state['section'].as_str
    topic = state['topic']
    resources = state['resources']
    draft = state['draft']
    queries = state['queries']
    num_revision = state['num_revision']

    prompt = ChatPromptTemplate([
        ('system', f'''
연구 리포트의 주제와 세부 섹션명이 주어집니다.

현재 작성된 섹션의 드래프트를 평가하세요.

이 글의 내용을 명확하고 유익하게 작성하기 위해,
검색된 결과 이외의 새로운 내용을 더 조사해야 하는지 판단하세요.

이후, 추가 검색을 위해 필요한 검색어 쿼리를 작성하세요.
{config['configurable']['num_search_queries']} 개 이하의 쿼리를 출력하세요.

해당 글은 충분히 완성도가 높아 추가 조사가 필요하지 않을 수도 있습니다.
그런 경우에는 'Good'을 출력하고, 추가 쿼리를 비워두세요.
'''),

('user',f'''
주제: {topic}

세부 섹션명: {section}


드래프트: {draft}''')
])
    # revision 개수 넘어가면 바로 END
    if num_revision >= config['configurable']['max_search_depth']:
         return Command(goto = END,
                       update = {'finished':True})


    evaluator_llm = init_chat_model(
        model_provider = config['configurable']['evaluator_provider'],
        model= config['configurable']['evaluator_model'],
        rate_limiter=rate_limiter,
        temperature=0.8,
    )


    chain = prompt | evaluator_llm.with_structured_output(Feedback)

    feedback = chain.invoke({})

    if feedback.grade =='Good':
        return Command(goto = END,
                       update = {'finished':True})
    else:
        return Command(goto = 'search_and_filter',
                       update= {'queries': feedback.queries,
                                'num_revision': num_revision+1})

Research Agent를 구성하는 Small Graph를 만듭니다.

In [ ]:
builder = StateGraph(ResearchState)
builder.add_node(generate_search_query)
builder.add_node(search_and_filter)
builder.add_node(write_section)
builder.add_node(refine_research)


builder.add_edge(START, 'generate_search_query')
builder.add_edge('generate_search_query', 'search_and_filter')
builder.add_edge('search_and_filter', 'write_section')
builder.add_edge('write_section', 'refine_research')

memory = MemorySaver()



researcher_graph = builder.compile(checkpointer=memory)

In [ ]:
researcher_graph

In [ ]:
t = Section(name='서론', description='1M Context Windows 모델의 시대를 연 구글 Gemini와 Llama 4 모델에 대해 설명합니다.', research=True, content='')

In [ ]:
test_research_state={
    'topic': '1M Context Windows 모델',
    'section': t,
    'num_revision':0
}

# Thread
thread = {'configurable':default, 'thread_id':'0'}

history = []
for event in researcher_graph.stream(test_research_state, thread, stream_mode="updates"):
    for status in event:
        print(f'# {status}')
        for key in event[status]:
            value = str(event[status][key])
            if len(value)>300:
                print(f'- {key}: {value[:300]}')
            else:
                print(f'- {key}: {value}')
        print('---------')
    history.append(event)

In [ ]:
load_dotenv('.env', override=True)

작성한 섹션 드래프트를 확인해 보겠습니다.

In [ ]:
from IPython.display import display
from IPython.display import Markdown
import textwrap

result = history[-2]['write_section']['draft']

def to_markdown(text):
  text = text.replace('•', '  *')
  return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))

to_markdown(result)


섹션별 Writer를 구성했습니다.   
이제 해당 그래프를 서브모듈로 하는 에이전트 구조를 구성합니다.

In [ ]:
class State(TypedDict):
    topic: str
    plan: ReportPlan
    result: str
    human_feedback:str
    finished_drafts: Annotated[list[str], operator.add]


요청을 받은 뒤, 초기 설정과 함께 부가 질문을 수행합니다.

In [ ]:
def initiate_report(state: State , config: RunnableConfig):

    prompt = ChatPromptTemplate([
        ('system','''
당신은 주어진 주제에 대한 연구 보고서의 초기 방향 설정을 위한 개요를 구성합니다.
최대한 최신의 지식과 인사이트를 활용하여야 하며, 사용자를 위한 맞춤형 보고서가 되어야 합니다.

주어진 주제에 대한 섹션별 개요를 작성하세요.
단, 마지막 섹션인 결론은 제외하고 작성하세요.

각 섹션은 불필요한 요소를 포함하지 말아야 하며, 명확하게 구분되어야 합니다.
섹션 간의 겹치는 내용을 최대한 줄이고, 각각의 역할이 분명하도록 구성하세요.

또한, 최종 결과 보고서에 사용자의 선호를 최대한 반영하기 위해, 사용자에게 추가로 질문할 내용도 작성하세요.
예를 들어, 세부 분야, 적용하고자 하는 환경, 원하는 정리 형식 등을 질문할 수 있습니다.

다음은 예시입니다.

---
질문: LLM 파인 튜닝 방법인 LoRA의 최근 발전된 모델들에 대해 조사해줘


답변:
개요: (보고서의 개요)
추가 질문: LoRA 기반 LLM 파인튜닝 관련 최근 발전된 모델들에 대해 조사해드릴게요.
아래 항목들 중 가능한 정보를 알려주시면 더 정확한 조사를 도와드릴 수 있어요:
용도 (예: 챗봇, 코드 생성, 번역, 의료 등)
적용 환경 (예: 연구용, 기업 서비스용, 모바일 디바이스 등)
원하시는 정리 형식 (예: 표, 요약 보고서, 논문 중심 정리 등)
가능한 범위를 알려주시면 곧바로 조사 시작할게요!'''),

('human', '''사용자의 주제(혹은 요청): {topic}

---

관련 최신 검색 결과:
{context}
 ''')
    ])

    topic = state['topic']

    search_tool =  tool_list[config['configurable']['search_api'][0]]
    # 적절한 검색 툴 선택 (여기서는 Tavily로 고정)


    context = search_tool.invoke(topic)
    # 초기 검색을 수행하여 개요 작성

    planner_llm = init_chat_model(
        model_provider = config['configurable']['planner_provider'],
        model= config['configurable']['planner_model'],
        rate_limiter=rate_limiter,
        temperature=0.8,
    ).with_structured_output(ReportPlan)

    chain = prompt | planner_llm

    result = chain.invoke({'topic': topic, 'context' : context})

    print('# Planner: ')

    print(f'''
# 보고서 작성 개요:
{result.as_str}

# 추가 질문:
{result.followup_question}''')
    return {'plan':result}


In [ ]:
def human_review(state: State , config: RunnableConfig):

    human_review = interrupt(
        {
            "question": "피드백을 전달해 주세요, 이대로 진행하고 싶으시면, continue 또는 go만 입력하세요.",
        }
    )
    # Human Feedback을 받아 전달
    review_action = human_review.get("human_feedback")
    return {'human_feedback': review_action}

In [ ]:
def refine_outline(state: State , config: RunnableConfig):
    current_plan = state['plan']
    human_feedback = state['human_feedback']

    if human_feedback.lower()=='go' or human_feedback.lower()=='continue':
        return {'plan': current_plan}

    refine_prompt = PromptTemplate(template='''
보고서의 개요가 주어집니다.
추가 요청사항을 반영하여, 수정된 개요를 작성하세요:

기존 개요:
{current_plan}

---

피드백:
{feedback}

    ''')

    planner_llm = init_chat_model(
        model_provider = config['configurable']['planner_provider'],
        model= config['configurable']['planner_model'],
        rate_limiter=rate_limiter,
        temperature=0.8,
    ).with_structured_output(ReportPlan)

    # 수정된 계획 생성
    refine_chain = refine_prompt | planner_llm

    refined_plan = refine_chain.invoke({
        'current_plan': current_plan.as_str,
        'feedback': human_feedback
    })

    print('# 수정된 보고서 계획:')
    print(f'''
    {refined_plan.as_str}

    추가 질문:
    {refined_plan.followup_question}
    ''')

    return {'plan': refined_plan}

In [ ]:
def research(state:ResearchState, config: RunnableConfig):
    result = researcher_graph.invoke({
    'topic': state['topic'],
    'section': state['section'],
    'num_revision':0
    })
    draft = result['draft']

    return {'finished_drafts':[draft]}


def start_survey(state:State, config: RunnableConfig):
    topic = state['topic']
    plan = state['plan']
    # Query 생성, 수집, Reflection, 섹션 작성 모듈을 하나의 에이전트로 구성

    return [Send("research",
            {'topic':topic, "section": s}) for s in plan.sections]


def synthesizer(state:State, config: RunnableConfig):
    return {'result':'\n'.join(state['finished_drafts'])}

def finalizer(state:State, config: RunnableConfig):
    prompt = PromptTemplate(template='''
연구 보고서의 내용이 주어집니다.
전체 흐름을 고려하여, 최종 결론 섹션을 작성하세요.

전체 보고서 내용:
{result}''')

    chain = prompt | llm | StrOutputParser()
    result = chain.invoke({'result':state['result']})
    return {'result':state['result'] + '\n'+result}


In [ ]:
builder = StateGraph(State)
builder.add_node(initiate_report)
builder.add_node(human_review)
builder.add_node(refine_outline)
builder.add_node(research)
builder.add_node(synthesizer)
builder.add_node(finalizer)

builder.add_edge(START, 'initiate_report')
builder.add_edge('initiate_report','human_review')
builder.add_edge('human_review', 'refine_outline')

builder.add_conditional_edges("refine_outline", start_survey, ["research"])

builder.add_edge('research', 'synthesizer')
builder.add_edge('synthesizer', 'finalizer')
builder.add_edge('finalizer',END)
memory = MemorySaver()



graph = builder.compile(checkpointer=memory)

In [ ]:
graph

In [ ]:
from IPython.display import Image, display

display(Image(graph.get_graph(xray=True).draw_mermaid_png()))

완성된 그래프를 실행해 보겠습니다.

In [ ]:
test_state = {'topic':'Long-Context LLM의 시대'}
thread = {'configurable':default, 'thread_id':'0'}

graph.invoke(test_state, config = thread)

In [ ]:
# BottleNeck: Search and Filter
# 긴 컨텍스트 모델은 필터링을 안 하는 방법도 고려할 수 있겠습니다..
'''좋아, Gemini 2.5 Pro는
긴 컨텍스트에서도 성능이 엄청 좋던데, 그 부분을 중요하게 다뤄 주고.
Llama 4의 Long Context에 대해서도 알려줘.        '''

history =[]

for event in graph.stream(
        Command(resume={"human_feedback": """섹션 개수를 2개로 줄여줘."""}),
    thread,
    stream_mode="updates", subgraphs=True
):
    history.append(event)
    for status in event:
        print(f'# {str(status)[:300]}')

        try:
            for key in event[status]:
                value = str(event[status][key])
                if len(value)>300:
                    print(f'- {key}: {value[:300]}')
                else:
                    print(f'- {key}: {value}')
            print('---------')
        except:
            continue

결과물을 md 파일에 저장해 보겠습니다.

In [ ]:
result = history[-1][1]['finalizer']['result']
result

In [ ]:
with open("example.md", "w", encoding="utf-8") as f:
    f.write(result)